In [2]:
import pandas as pd
import numpy as np
Billing = pd.read_csv("Billing.csv")
DiseaseCase = pd.read_csv("DiseaseCase.csv")
Lab = pd.read_csv("LabReduced.csv", dtype={"Name_calc": str, "CodeType_orig": str, "CodeType_calc": str, "Code_orig": str, "Code_calc": str})
Exam = pd.read_csv("Exam.csv")
Risk = pd.read_csv("RiskFactor.csv")
Patient = pd.read_csv("Patient.csv")
csvs = [Lab, Exam]

c:\users\susu1\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
a1c = Lab.loc[Lab["Code_calc"] == '17856-6']
a1c['TestResult_calc'] = a1c['TestResult_calc'].str.replace('\W', '')
a1c['TestResult_calc'] = a1c['TestResult_calc'].astype(float)

BP = Exam.loc[Exam["Exam1"] == 'sBP (mmHg)']
BP['Result1_calc'] = BP['Result1_calc'].replace('\W', '')
BP['Result1_calc'] = BP['Result1_calc'].astype(float)    
BP['Result2_calc'] = BP['Result1_calc'].replace('\W', '')
BP['Result2_calc'] = BP['Result1_calc'].astype(float) 

LDL = Lab.loc[Lab["Code_calc"] == '22748-8'] 
HDL = Lab.loc[Lab["Code_calc"] == '14646-4'] 
Tri = Lab.loc[Lab["Code_calc"] == '14927-8']

LDL['TestResult_calc'] = LDL['TestResult_calc'].str.replace('\W', '')
LDL['TestResult_calc'] = LDL['TestResult_calc'].astype(float)
HDL['TestResult_calc'] = HDL['TestResult_calc'].str.replace('\W', '')
HDL['TestResult_calc'] = HDL['TestResult_calc'].astype(float) 
Tri['TestResult_calc'] = Tri['TestResult_calc'].str.replace('\W', '')
Tri['TestResult_calc'] = Tri['TestResult_calc'].astype(float) 

alb = Lab.loc[Lab["Code_calc"] == '32294-1']
alb['TestResult_calc'] = alb['TestResult_calc'].str.replace('\W', '')
alb['TestResult_calc'] = alb['TestResult_calc'].astype(float) 

<ipython-input-3-b18ee225f20d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a1c['TestResult_calc'] = a1c['TestResult_calc'].str.replace('\W', '')
<ipython-input-3-b18ee225f20d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a1c['TestResult_calc'] = a1c['TestResult_calc'].astype(float)
<ipython-input-3-b18ee225f20d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [4]:
merge1 = LDL.merge(HDL,how='outer', on=['Patient_ID','PerformedDate', 'Name_calc', 'Code_calc', 'TestResult_calc', 'UnitOfMeasure_calc'])
merge2 = merge1.merge(Tri, how = 'outer', on=['Patient_ID','PerformedDate', 'Name_calc', 'Code_calc', 'TestResult_calc', 'UnitOfMeasure_calc'] )


In [5]:
header_list = ['Patient_ID','DateCreated','Exam1', 'd', 'Result1_calc', 'UnitOfMeasure_calc']
header2_list = ['Patient_ID','DateCreated','Exam2', 'd', 'Result2_calc', 'UnitOfMeasure_calc']
BPsys = BP.reindex(columns = header_list)
BPdia = BP.reindex(columns = header2_list)
BPdia['Exam1'] = BPdia['Exam2']
BPdia['Result1_calc'] = BPdia['Result2_calc']
BPnew = BPsys.append(BPdia)
BPnew = BPnew.drop(['Exam2', 'Result2_calc'], axis = 1)
BPnew = BPnew.rename(columns={"DateCreated": "PerformedDate", "Exam1": "Name_calc","d": "Code_calc", "Result1_calc": "BP_res"})
#print(BPnew)

In [7]:
albLip = alb.merge(merge2, how='outer', on=['Patient_ID','PerformedDate', 'Name_calc', 'Code_calc', 'UnitOfMeasure_calc'])
albLip = albLip.rename(columns={"TestResult_calc_x": "Albumin/Creatinine_Res", "TestResult_calc_y" : "Lipids_Res"})
bigDF = albLip.merge(a1c,how='outer', on=['Patient_ID','PerformedDate', 'Name_calc', 'Code_calc', 'UnitOfMeasure_calc'])
bigDF = bigDF.rename(columns={"TestResult_calc": "a1c_Res"})
bigDF = bigDF.append(BPnew)
#print(bigDF)
                                

In [8]:
bigDF['A1C_RF'] = 0
bigDF.loc[(bigDF["Code_calc"] == '17856-6') & (bigDF['a1c_Res'] < 7), 'A1C_RF'] = 1
bigDF.loc[(bigDF["Code_calc"] == '17856-6') & (bigDF['a1c_Res'] == 7), 'A1C_RF'] = 2
bigDF.loc[(bigDF["Code_calc"] == '17856-6') & (bigDF['a1c_Res'] > 7), 'A1C_RF'] = 3
#print(bigDF)

In [9]:
#Systolic
bigDF['BPsys_RF'] = 0
bigDF.loc[(bigDF['Name_calc'] == 'sBP (mmHg)') & (bigDF['BP_res'] <= 120 ), 'BPsys_RF'] = 1
bigDF.loc[(bigDF['Name_calc'] == 'sBP (mmHg)') & (bigDF['BP_res'] <= 130 ) & (bigDF['BP_res'] > 120), 'BPsys_RF'] = 2
bigDF.loc[(bigDF['Name_calc'] == 'sBP (mmHg)') & (bigDF['BP_res'] > 130 ), 'BPsys_RF'] = 3

#Diastolic
bigDF['BPdia_RF'] = 0
bigDF.loc[(bigDF['Name_calc'] == 'dBP (mmHg)') & (bigDF['BP_res'] <= 70 ), 'BPsys_RF'] = 1
bigDF.loc[(bigDF['Name_calc'] == 'dBP (mmHg)') & (bigDF['BP_res'] <= 80 ) & (bigDF['BP_res'] > 70 ), 'BPsys_RF'] = 2
bigDF.loc[(bigDF['Name_calc'] == 'dBP (mmHg)') & (bigDF['BP_res'] > 80 ), 'BPsys_RF'] = 3

bigDF['BP_RF'] = bigDF[['BPsys_RF', 'BPdia_RF']].mean(axis=1)
#print(bigDF)

In [10]:
#LDL
bigDF['LDL_RF'] = 0
bigDF.loc[(bigDF["Code_calc"] == '22748-8') & (bigDF['Lipids_Res'] <= 5.6 ), 'LDL_RF'] = 1
bigDF.loc[(bigDF["Code_calc"] == '22748-8') & (bigDF['Lipids_Res'] >5.6) & (bigDF['Lipids_Res'] < 7.2), 'LDL_RF'] = 2
bigDF.loc[(bigDF["Code_calc"] == '22748-8') & (bigDF['Lipids_Res'] >= 7.2 ), 'LDL_RF'] = 3

#HDL
bigDF['HDL_RF'] = 0
bigDF.loc[(bigDF["Code_calc"] == '14646-4') & (bigDF['Lipids_Res'] => 3.3 ), 'HDL_RF'] = 1
bigDF.loc[(bigDF["Code_calc"] == '14646-4') & (bigDF['Lipids_Res'] < 3.3 ) & (bigDF['Lipids_Res'] > 2.2), 'HDL_RF'] = 2
bigDF.loc[(bigDF["Code_calc"] == '14646-4') & (bigDF['Lipids_Res'] <= 2.2 ), 'HDL_RF'] = 3

#Tri
bigDF['Tri_RF'] = 0
bigDF.loc[(bigDF["Code_calc"] == '14927-8') & (bigDF['Lipids_Res'] <= 1.7 ), 'Tri_RF'] = 1
bigDF.loc[(bigDF["Code_calc"] == '14927-8') & (bigDF['Lipids_Res'] > 1.7 ) & (bigDF['Lipids_Res'] < 5.6), 'Tri_RF'] = 2
bigDF.loc[(bigDF["Code_calc"] == '14927-8') & (bigDF['Lipids_Res'] >= 5.6 ), 'Tri_RF'] = 3

bigDF['Lip_RF'] = bigDF[['LDL_RF', 'HDL_RF', 'Tri_RF']].mean(axis=1)
#print(bigDF)

In [11]:
#Albumin
bigDF['Alb_RF'] = 0
bigDF.loc[(bigDF["Code_calc"] == '32294-1') & (bigDF['Albumin/Creatinine_Res'] <= 3.39 ), 'Alb_RF'] = 1
bigDF.loc[(bigDF["Code_calc"] == '32294-1') & (bigDF['Albumin/Creatinine_Res'] > 3.39 ) & (bigDF['Albumin/Creatinine_Res'] < 33.9), 'Alb_RF'] = 2
bigDF.loc[(bigDF["Code_calc"] == '32294-1') & (bigDF['Albumin/Creatinine_Res'] >= 33.9 ), 'Alb_RF'] = 3
#print(bigDF)

In [12]:
bigDF['Total_RF'] = bigDF[['A1C_RF', 'BP_RF', 'Lip_RF', 'Alb_RF']].sum(axis=1)
#print(bigDF)

In [13]:
bigDF = bigDF.drop(['Code_calc','UnitOfMeasure_calc','BPsys_RF' ,'BPdia_RF','LDL_RF', 'HDL_RF', 'Tri_RF'], axis = 1)
print(bigDF)

           Patient_ID PerformedDate                             Name_calc  \
0        1.000100e+12    2017-11-02  Urine Albumin/Creatinine Ratio (ACR)   
1        1.000100e+12    2017-10-16  Urine Albumin/Creatinine Ratio (ACR)   
2        1.000100e+12    2014-04-07  Urine Albumin/Creatinine Ratio (ACR)   
3        1.000100e+12    2014-03-07  Urine Albumin/Creatinine Ratio (ACR)   
4        1.000100e+12    2014-02-05  Urine Albumin/Creatinine Ratio (ACR)   
...               ...           ...                                   ...   
1048566  4.000000e+12    2015-08-17                            dBP (mmHg)   
1048570  4.000000e+12    2016-09-21                            dBP (mmHg)   
1048571  4.000000e+12    2015-07-17                            dBP (mmHg)   
1048572  4.000000e+12    2018-01-22                            dBP (mmHg)   
1048573  4.000000e+12    2017-08-30                            dBP (mmHg)   

         Albumin/Creatinine_Res  Lipids_Res  a1c_Res  BP_res  A1C_RF  BP_RF

In [14]:
##exports to zipped csv file
compression_opts = dict(method='zip',
                        archive_name='final.csv')  
bigDF.to_csv('final.zip', index=False,
          compression=compression_opts)  

# Generate risk score based on vector of lab data

In [ ]:
#lab = ['09/19/18', 13, 122, 88, 40, 8, 6, 17.5]
#assuming vector is lab = [date, a1c, BP sys, BP dias, LDL, HDl, Tri, alb]
def riskScoreCalc(lab):
    #a1c
    a1c = lab[1]
    a1cRF = 0
    if a1c>7: 
        a1cRF = 3
    elif a1c==7: 
        a1cRF = 2
    elif a1c<7:
        a1cRF = 1
    #BP RF
    bpSys = lab[2]
    bpSysRF = 0
    bpDiaRF = 0
    if bpSys > 130:
        bpSysRF = 3
    elif bpSys <= 120:
        bpSysRF = 1
    elif (bpSys > 120) and (bpSys <= 130):
        bpSysRF = 2
    bpDia = lab[3]
    if bpDia > 80:
        bpDiaRF = 3
    elif bpDia <= 70:
        bpDiaRF = 1
    elif (bpDia > 70) and (bpDia <= 80):    
        bpDiaRF = 2
    
    BPsum = bpSysRF + bpDiaRF
    BP_RF = BPsum/2
    
    #Lipids
    ldl = lab[4]
    hdl = lab[5]
    tri = lab[6]
    ldlRF = 0
    hdlRF = 0 
    triRF = 0
    if ldl >= 7.2:
        ldlRF = 3
    elif ldl <= 5.6:
        ldlRF = 1
    elif (ldl > 5.6) and (ldl < 7.2):
        ldlRF = 2
    if hdl >= 3.3:
        hdlRF = 1
    elif hdl <= 2.2:
        hdlRF = 3
    elif (hdl > 2.2) and (hdl < 3.3):
        hdlRF = 2
    if tri >= 5.6:
        triRF = 3
    elif tri <= 1.7:
        triRF = 1
    elif (tri > 1.7) and (tri < 5.6):
        triRF = 2
    
    lipsum = ldlRF + hdlRF + triRF
    lipRF = lipsum/3
    
    #alb
    alb = lab[7]
    albRF = 0 
    if alb >= 33.9:
        albRF = 3
    elif alb <= 3.39:
        albRF = 1
    elif (alb > 3.39) and (alb < 33.9):
        albRF = 2
        
    #totalRF
    totalRF = a1cRF + BP_RF + lipRF + albRF
    
    ##return vector with date, individual risk scores and total RF
    date = lab[0]
    risk = [date, a1cRF, BP_RF, lipRF, albRF, totalRF]
    print(risk)

riskScoreCalc(lab)

# Carrying values from visits within 2 weeks of each other
a work in progress lol

In [ ]:
#This takes a while to run and I didn't notice a difference
#bigDF['PerformedDate'] = bigDF['PerformedDate'].apply(pd.to_datetime)

In [ ]:
#bigDF = bigDF.sort_values(by='PerformedDate',ascending=True)
#uniqID = bigDF.Patient_ID.unique()
#print(uniqID)

In [ ]:
#column_names = ['Patient_ID','PerformedDate', 'Name_calc', 'Code_calc', 'Albumin/Creatinine_Res','UnitOfMeasure_calc' ,'Lipids_Res', 'a1c_Res', 'BP_res']
#newDF = pd.DataFrame(columns = column_names)
#print(newDF)
#from datetime import datetime, timedelta

In [ ]:
#visits within 2 weeks of each other
#for i in uniqID:   #for i in list of unique patient_IDs
#    df = bigDF.loc[bigDF["Patient_ID"] == i]      #make a dataframe with information(performed date, exam results) from each of their visits
#    df = df.sort_values(by='PerformedDate',ascending=True)   #sort the rows by PerformedDate in ascending order so consecutive visits are next to each other
#    for j in df:    
#        df['delta'] = (df['PerformedDate']-df['PerformedDate'].shift())  #create a column that stores the difference in dates between consecutive visits
#        df = df.dropna(subset = ['delta'])    #drop any NaN values in the difference in dates (some dates had NULL values)
#        for index, row in df.iterrows():
#            temp = (row['delta'])
#            if temp < timedelta(days = 14):   #if the time difference between the current visit and the next visit is <2 weeks 
#               newDF = newDF.append(df.loc[index])  #copy the information to a new dataframe
#               print(newDF)                        